In [1172]:
# Principales
import pandas as pd
import numpy as np

# Modificaciones 
import re
from unidecode import unidecode
import unicodedata
import datetime
import os


# visuzalizaciones
import missingno as msno
import matplotlib as plt 
import plotly.express as px 

In [1173]:
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows',20)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_colwidth', None)
# Ajustar la configuración para mostrar más filas 
#pd.set_option('display.max_rows', 500)

#inicializo la lista to replace
to_replace=['Null','Nan','nan','NULL',None]

# Funciones

In [1174]:


# Función para normalizar texto
def normalize_text(text):
    # Reemplazar caracteres especiales y convertir a minúsculas para una comparación consistente
    text = text.replace("/", " ")  # Reemplazar la barra por espacio o eliminarla completamente
    text = text.replace("ñ", "n")  # Reemplazar la "ñ" por "n"
    return ''.join(
        c for c in unicodedata.normalize('NFD', text)
        if unicodedata.category(c) != 'Mn'
    ).lower()

In [1175]:
def actualizar_valores(df, df1, clave, columna_a_actualizar):
    """
    Actualiza los valores de una columna en un dataframe utilizando los valores de otro dataframe.
    
    :param df: DataFrame a actualizar.
    :param df1: DataFrame que contiene los valores correctos.
    :param clave: Nombre de la columna que funciona como clave primaria para hacer la correspondencia.
    :param columna_a_actualizar: Nombre de la columna que será actualizada en df.
    :return: DataFrame actualizado.
    """
    # Crear un diccionario con los valores correctos del segundo dataframe
    valores_correctos = df1.set_index(clave)[columna_a_actualizar].to_dict()
    
    # Actualizar los valores en el dataframe principal
    df[columna_a_actualizar] = df[clave].map(valores_correctos).fillna(df[columna_a_actualizar])
    
    return df


In [1176]:
def porcentaje_nulos(df, columnas):
    if isinstance(columnas, str):
        columnas = [columnas]  # Convertir a lista si es una sola columna
    resultados = {}
    for columna in columnas:
        if columna in df.columns:
            total_filas = len(df)
            nulos = df[columna].isnull().sum()
            porcentaje_nulos = (nulos / total_filas) * 100
            resultados[columna] = porcentaje_nulos
        else:
            resultados[columna] = 'Column not found'
    return resultados



# Carga de datos

In [1177]:

print(os.getcwd())

c:\Users\extas\Documents\GitHub\precios-de-vivienda-madrid\codigo\01-preprocesamiento-de-datos\Revision_columnas


In [1178]:
df17= pd.read_csv('./data/dfpisosCompleto17012025.csv',index_col=False)

C:\Users\extas\AppData\Local\Temp\ipykernel_11668\2425994981.py:1: DtypeWarning: Columns (5,7,16,28,32,70,71,76,82) have mixed types. Specify dtype option on import or set low_memory=False.
  df17= pd.read_csv('./data/dfpisosCompleto17012025.csv',index_col=False)


In [1179]:
df17 = df17.loc[:, ~df17.columns.str.contains('^Unnamed')]

In [1180]:
df17.shape

(10529, 110)

In [1181]:
df17.columns.tolist()

['title',
 'url',
 'precio',
 'precio_anterior',
 'descuento',
 'EUR/m2',
 'm2_constr',
 'distrito',
 'cod_distrito',
 'barrio',
 'cod_barrio',
 'zona',
 'calle',
 'detalle',
 'consumoce_ano',
 'letrace',
 'emisiones_co2',
 'tipologia',
 'propiedad_completa',
 'estado',
 'amueblado',
 'planta',
 'habitaciones',
 'dormitorios',
 'banos',
 'balcon',
 'terraza',
 'garaje',
 'trastero',
 'ascensor',
 'porteria',
 'calefaccion',
 'ano_construccion',
 'plantas_edificio',
 'aire_acondicionado',
 'jardin',
 'inmueble_ingresos',
 'alquiler_opcion_a_compra',
 'disponibilidad',
 'nuda_propiedad',
 'sup_comercial',
 'tipo_inmueble',
 'tiene_armario',
 'cancha_tenis',
 'carpinteria_exterior_doble_vidrio/pvc',
 'carpinteria_exterior_doble_vidrio/madera',
 'carpinteria_exterior_doble_vidrio/metal',
 'carpinteria_exterior_triple_vidrio/madera',
 'carpinteria_exterior_triple_vidrio/metal',
 'carpinteria_exterior_vidrio/pvc',
 'carpinteria_exterior_vidrio/madera',
 'carpinteria_exterior_vidrio/metal',
 

In [1182]:
df17['cantidad_visitas'] = np.where((df17['cantidad_visitas'].isnull()) | (df17['cantidad_visitas'] == 0), df17['n.visitas'], df17['cantidad_visitas'])


In [1183]:
df17['n.visitas'].value_counts(dropna=False)

n.visitas
NaN       9647
30.00      153
40.00      139
20.00      130
50.00       97
          ... 
170.00       2
140.00       1
200.00       1
190.00       1
180.00       1
Name: count, Length: 21, dtype: int64

In [1184]:
# Guardar el DataFrame en un archivo CSV sin incluir el índice
#df.to_csv("dfpisosCompleto17012025.csv", index=False)


In [1185]:
df17["porteria"].value_counts(dropna=False)

porteria
NaN                      10372
Portero todo el día        153
Portero media jornada        4
Name: count, dtype: int64

# revision columnas

In [1186]:


# Supongamos que df17 es tu DataFrame
#df17['porteria'] = df17['porteria'].replace('0', np.nan)
df17['calefaccion'] = df17['calefaccion'].replace('0.0', np.nan)
df17['calefaccion'] = df17['calefaccion'].replace('Climatización', 'climatizacion')
df17['calefaccion'] = df17['calefaccion'].replace('Climatizaciòn', 'climatizacion')
df17['tiene_armario'] = df17['tiene_armario'].replace(0.0, np.nan)
df17.loc[df17['web'].isnull(), 'web'] = 'Indomio.com'
df17['tiene_piscina'] = df17['tiene_piscina'].replace(0, np.nan)


df17['porteria'] = df17['porteria'].replace('1', 'Portero todo el día')


## calefaccion

In [1187]:
df17["calefaccion"].value_counts(dropna=False)

calefaccion
NaN                        6949
Central                     927
Individual                  646
1.0                         578
Gas natural                 520
                           ... 
Por climatizacion             1
Contadores individuales       1
Bomba de calor                1
Conducto                      1
Gas natural y biomasa         1
Name: count, Length: 62, dtype: int64

In [1188]:


# Palabras clave para clasificar
central_keywords = ["central", "comunitaria", "comunitario"]
individual_keywords = ["individual", "particular", "chimenea", "bomba", "termo", "climatizacion"]
electric_keywords = ["electrica", "bomba", "climatización", 'termo electrico','climatizacion']
gas_keywords = ["gas", "propano", "g n", "g/n"]  # Asegurarse de que 'g n' y 'g/n' estén bien escritos
gasoil_keywords = ["gasóil","gasoil",'gasoil',"Gasoil"]  # Añadir "gasoil" aquí
renovables_keywords = ["aerotermia", "geotermia", "chimenea"]
suelo_radiante_keywords=["suelo_radiante", "suelo"]
#radiadores_keywords=["radiadores"]
aire_keywords=["bomba","climatización",'climatizacion','conducto']
desconocido_keywords=['chimenea', 'termo electrico']
#lo mas extendido son los radiadores por eso despues de pasar , suelo radiante y aire pongo a todos radiadores salvo a chimenea y termo electrico que no lo se
radiadores_keywords=['radiadores','Gas natural', 'G/n', 'Gas particular', 'Central, alimentación eléctrica', 'Agua caliente y calefacción central con contador i', 'Comunitario', 'Individual con gas natural', 'Individual, de gasóil', 'Comunitaria', 'Calefaccion Propano', 'Calefaccion Gas Natural', 'Central, por aire, de gas', 'Gas natural y biomasa', 'Centralizada', 'Eléctrica', 'Calefaccion Individual', 'Calefaccion individual','Contadores individuales', '1.0', 'Individual, alimentación eléctrica', 'Aerotermia', 'Individual - gas natural', 'Individual', 'Calefaccion Aerotermia', 'Gasoil', 'Central individualizada', 'Individual, por aire, alimentación eléctrica', 'Geotermia y aerotermia', 'Calefaccion comunitaria', 'Calefaccion Central', 'Central, de gas', 'Calefaccion Electrica', 'Calefacción central individualizada', 'Central, de gasóil', 'Individual, de gas', 'Central']

# Inicializar conjuntos para valores únicos
central = set()
individual = set()
electrica = set()
gas = set()
gsoil = set()
renovables = set()
suelo_radiante = set()
radiadores = set()
aire = set()
sin_clasificar = set()
sin_clasificar_energia = set()
sin_clasificar_medio = set()
radiadores=set()
suelo_radiante=set()


# Clasificar categorías
for categoria in df17["calefaccion"]:
    categoria_normalized = normalize_text(str(categoria))  # Normalizar texto
    
 
    
    # Clasificación por tipo (central o individual)
    if any(keyword in categoria_normalized for keyword in central_keywords):
        central.add(categoria)
    elif any(keyword in categoria_normalized for keyword in individual_keywords):
        individual.add(categoria)
    else:
        sin_clasificar.add(categoria)
    
    # Clasificación por energía (eléctrica, gas, gasoil)
    if any(keyword in categoria_normalized for keyword in electric_keywords):
        electrica.add(categoria)
    elif any(keyword in categoria_normalized for keyword in gasoil_keywords):
        gsoil.add(categoria)
    elif any(keyword in categoria_normalized for keyword in gas_keywords):
        gas.add(categoria)
      
   
    elif any(keyword in categoria_normalized for keyword in renovables_keywords):
        renovables.add(categoria)
    else:
        sin_clasificar_energia.add(categoria)

    # Clasificación por medio (suelo radiante, radiadores, aire)
    
   
    if any(keyword in categoria_normalized for keyword in aire_keywords):
        aire.add(categoria)    
    elif any(keyword in categoria_normalized for keyword in suelo_radiante_keywords):
        suelo_radiante.add(categoria)
    elif any(keyword in categoria_normalized for keyword in desconocido_keywords):
        sin_clasificar_medio.add(categoria)
  #  else: any(keyword in categoria_normalized for keyword in radiadores_keywords):
    else: 
        radiadores.add(categoria)
    

# Convertir los conjuntos a listas
central = list(central)
individual = list(individual)
sin_clasificar = list(sin_clasificar)
electrica = list(electrica)
gas = list(gas)
gasoil = list(gsoil)
renovables = list(renovables)
sin_clasificar_energia = list(sin_clasificar_energia)
aire=list(aire)
radiadores=list(radiadores)
suelo_radiante=list(suelo_radiante)
sin_clasificar_medio=list(sin_clasificar_medio)
# Imprimir resultados
print("Central:", central)
print("Individual:", individual)
print("No clasificado:", sin_clasificar)
print("Electrica:", electrica)
print("Gas:", gas)
print("Gasoil:", gasoil)
print("Renovables:", renovables)
print("No clasificado en energía:", sin_clasificar_energia)
print("Suelo Radiante:", suelo_radiante)
print("Radiadores:", radiadores)
print("Aire:", aire)
print("No clasificado en medio:", sin_clasificar_medio)


Central: ['Central, de gasóil', 'Central, de gas', 'Central individualizada', 'Comunitaria', 'Central, alimentación eléctrica', 'Central, suelo radiante, de gas', 'Calefacción central individualizada', 'Centralizada', 'Calefaccion comunitaria', 'Central', 'Central, por aire, de gas', 'Central, suelo radiante', 'Central, por radiadores, de gasóil', 'Comunitario', 'Calefaccion Central', 'Agua caliente y calefacción central con contador i', 'Central, por radiadores', 'Central, por radiadores, de gas']
Individual: ['Calefaccion Individual', 'Bomba frio-calor', 'Individual, suelo radiante, de gas', 'Individual, suelo radiante', 'Individual, de gasóil', 'Individual, por aire, por bomba de calor', 'Individual, por radiadores, alimentación eléctrica', 'Termo electrico', 'Individual, de gas', 'Bomba frio/calor', 'Bomba de calor', 'Individual, por radiadores', 'Individual', 'Individual, por radiadores, de gasóil', 'Individual - gas natural', 'Contadores individuales', 'Bomba frio calor', 'Chimen

In [1189]:
df17["ano_construccion"].value_counts(dropna=False)

ano_construccion
NaN                     6994
Más de 50 años          1018
Entre 30 y 50 años       205
 Mas de 50 anos          131
1900.0                   118
                        ... 
 Entre 10 y  15 Anos       1
 1949                      1
 1943                      1
 1928                      1
1944.0                     1
Name: count, Length: 188, dtype: int64

In [1190]:
df17["plantas_edificio"].value_counts(dropna=False)

plantas_edificio
NaN      10295
6.00        45
4.00        42
3.00        32
7.00        25
5.00        24
8.00        21
2.00        14
10.00       14
9.00         8
1.00         5
12.00        2
23.00        1
11.00        1
Name: count, dtype: int64

In [1191]:
df17.columns = df17.columns.str.strip()

In [1192]:
df17.columns

Index(['title', 'url', 'precio', 'precio_anterior', 'descuento', 'EUR/m2',
       'm2_constr', 'distrito', 'cod_distrito', 'barrio',
       ...
       'cantidad_dormitorios', 'orientacion_este', 'orientacion_norte',
       'orientacion_norte,_sur', 'orientacion_oeste', 'orientacion_sur',
       'orientacion_sur,_este', 'sistema_alarma', 'n.visitas', 'grupo_cocina'],
      dtype='object', length=110)

In [1193]:
df17["plantas_edificio"].value_counts(dropna=False)

plantas_edificio
NaN      10295
6.00        45
4.00        42
3.00        32
7.00        25
5.00        24
8.00        21
2.00        14
10.00       14
9.00         8
1.00         5
12.00        2
23.00        1
11.00        1
Name: count, dtype: int64

## aire acondicionado

In [1194]:
df17["aire_acondicionado"].value_counts(dropna=False)

aire_acondicionado
NaN                                                 5976
0                                                   1038
0.0                                                  845
Individual                                           645
Frío                                                 480
                                                    ... 
Aire frio/ calor por conductos/suelo refrescante       1
No disponible                                          1
Centralizada, frío                                     1
Por conductos en toda la casa                          1
Climatizacion por conductos frio y calor               1
Name: count, Length: 36, dtype: int64

In [1195]:
df17["aire_acondicionado"].unique()


array([nan, 'Preinstalación', 'Individual, frío/calor',
       'Individual, frío', 'Individual', 'Frío/calor',
       'Centralizada, frío', 'Centralizada, frío/calor',
       'Centralizada, calor', 'Centralizada', 'Frío', 'Frío y calor',
       'Splites repartidos', 'Por conductos en toda la casa',
       'Aerotermia', 'Independiente', 'Con bomba de calor', 'A/a f/c',
       'Suelo refrigerante', 'Preinstalado', 'Frío-calor',
       'Suelo radiante refrescante', 'Sistema air zone',
       'Climatizacion por conductos frio y calor',
       'Frió/calor por conductos', 'Central', 'Suelo radiante',
       'Tiene la instalción hecha en toda la casa por cond',
       'Geotermia y aerotermia', 'En toda la casa, por conductos',
       'Aire frio/ calor por conductos/suelo refrescante',
       'No disponible', '1', '0', '0.0', '1.0'], dtype=object)

In [1196]:
# Listas de valores para cada grupo
eficientes_val_columna = [
    'Aerotermia', 'Geotermia y aerotermia', 'Suelo radiante refrescante', 'Suelo refrigerante',
    'Sistema air zone', 'Climatizacion por conductos frio y calor', 'Frió/calor por conductos', 
    'Suelo radiante', 'Aire frio/ calor por conductos/suelo refrescante', 'Por conductos en toda la casa', 
    'Tiene la instalción hecha en toda la casa por cond', 'En toda la casa, por conductos'
]

normales_val_columna = [
    'Individual, frío/calor', 'Individual, frío', 'Individual', 'Frío/calor', 'Centralizada, frío', 
    'Centralizada, frío/calor', 'Centralizada, calor', 'Central', 'Frío', 'Frío y calor', 
    'Splites repartidos', 'Independiente', 'Con bomba de calor', 'A/a f/c', 
    'Aire frio/calor por conductos/suelo refrescante', '1', '1.0'
]

sin_aire_val_columna = ['Preinstalación', 'Preinstalado', 'No disponible', '0', '0.0']

# Función para clasificar los sistemas
def clasificar_sistema(valor):
    if valor in eficientes_val_columna:
        return 'Eficiente'
    elif valor in normales_val_columna:
        return 'Normal'
    elif valor in sin_aire_val_columna:
        return 'Sin Aire'
    else:
        return 'Desconocido'  # Si el valor no coincide con ninguno de los grupos




In [1197]:
# Crear una nueva columna en el DataFrame
df17['clasificacion_aire'] = df17['aire_acondicionado'].apply(clasificar_sistema)


In [1198]:
df17["jardin"].unique()

array([nan, 'Privado', 'Sin jardín', 'Comunitario', '0.0', '1.0'],
      dtype=object)

In [1199]:
# Ejemplo de uso con una sola columna
porcentaje = porcentaje_nulos(df17, 'plantas_edificio')
print(f"Porcentaje de valores nulos en 'plantas_edificio': {porcentaje.get('plantas_edificio', 'Column not found'):.2f}%")


Porcentaje de valores nulos en 'plantas_edificio': 97.78%


In [1200]:

# Ejemplo de uso
columnas = ["inmueble_ingresos", "alquiler_opcion_a_compra", "disponibilidad","nuda_propiedad","sup_comercial"]  # Reemplaza con las columnas de tu DataFrame
porcentaje_resultados = porcentaje_nulos(df17, columnas)

# Mostrar los resultados
for columna, porcentaje in porcentaje_resultados.items():
    print(f"Porcentaje de nulos en {columna}: {porcentaje:.2f}%")

Porcentaje de nulos en inmueble_ingresos: 99.92%
Porcentaje de nulos en alquiler_opcion_a_compra: 99.97%
Porcentaje de nulos en disponibilidad: 95.55%
Porcentaje de nulos en nuda_propiedad: 99.98%
Porcentaje de nulos en sup_comercial: 96.46%


In [1201]:
df17["disponibilidad"].value_counts(dropna=False)

disponibilidad
NaN      10060
Libre      469
Name: count, dtype: int64

La columna disponibilidad no aporta nada

In [1202]:
df17["alquiler_opcion_a_compra"].value_counts(dropna=False)

alquiler_opcion_a_compra
NaN     10526
1.00        3
Name: count, dtype: int64

In [1203]:
df17.query("(inmueble_ingresos == 1) | (alquiler_opcion_a_compra == 1) | (nuda_propiedad == 1)").count()


title                    13
url                      13
precio                   13
precio_anterior           0
descuento                 0
                         ..
orientacion_sur,_este     0
sistema_alarma            2
n.visitas                 0
grupo_cocina             13
clasificacion_aire       13
Length: 111, dtype: int64

## sup_comercial

In [1204]:
df17[df17['sup_comercial'].notnull()].head()

,title,url,precio,precio_anterior,descuento,EUR/m2,m2_constr,distrito,cod_distrito,barrio,cod_barrio,zona,calle,detalle,consumoce_ano,letrace,emisiones_co2,tipologia,propiedad_completa,estado,amueblado,planta,habitaciones,dormitorios,banos,balcon,terraza,garaje,trastero,ascensor,porteria,calefaccion,ano_construccion,plantas_edificio,aire_acondicionado,jardin,inmueble_ingresos,alquiler_opcion_a_compra,disponibilidad,nuda_propiedad,sup_comercial,tipo_inmueble,tiene_armario,cancha_tenis,carpinteria_exterior_doble_vidrio/pvc,carpinteria_exterior_doble_vidrio/madera,carpinteria_exterior_doble_vidrio/metal,carpinteria_exterior_triple_vidrio/madera,carpinteria_exterior_triple_vidrio/metal,carpinteria_exterior_vidrio/pvc,carpinteria_exterior_vidrio/madera,carpinteria_exterior_vidrio/metal,chimenea,cocina,fibra_optica,instalacion_tv_centralizada,instalacion_tv_individual,exterior,interior,interior_y_exterior,tiene_piscina,porton_electrico,puerta_blindada,alarma,videoportero,lujosa,descripcion_extendida,€_comunidad_mes,acceso_minusvalido,orientacion,web,fecha_descarga,certificado_energetico,piscina,tiene_jardin,barrios,cod_bario,referencia,cantidad_visitas,fecha_publicacion,titular_anuncio,campo12,cantidad_armarios,consumo_energia,cal_energetica,agua_caliente,tipo_suelo,m2_utiles,vista_zona,gastos_comunidad,zonas_verdes,terrazas,puerta_seguridad,num_plantas,num_pisos,reservado,nombre_oficina,cod_barrio_2,codigo_Postal,tipo_fachada,cantidad_dormitorios,orientacion_este,orientacion_norte,"orientacion_norte,_sur",orientacion_oeste,orientacion_sur,"orientacion_sur,_este",sistema_alarma,n.visitas,grupo_cocina,clasificacion_aire
2,"Casa plurifamiliar Plaza de Italia, Legazpi, Madrid",https://www.indomio.es/anuncios/81008864/,200000.00,NaN,NaN,500.00,400.00,Arganzuela,2.00,Legazpi,24.00,Legazpi,plaza de italia,Casa plurifamiliar Plaza de Italia,desconocido,NaN,No indicado,Casa,NaN,NaN,NaN,Sótano (-5),NaN,0.00,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,23.00,NaN,Privado,NaN,NaN,NaN,NaN,401.00,NaN,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0.00,Escribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorio,NaN,NaN,NaN,Indomio.com,2024-11-20,No indicado,-,1.00,NaN,24.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sin_informacion,Desconocido
18,"Piso de tres habitaciones buen estado, Ensanche de Vallecas-Valdecarros, Madrid",https://www.indomio.es/anuncios/87841991/,450000.00,NaN,NaN,4.59,98.00,Villa de Vallecas,18.00,EnsanchedeVallecas,183.00,Ensanche de Vallecas,NaN,Piso de tres habitaciones buen estado,desconocido,NaN,No indicado,Piso,NaN,Bueno / Habitable,No,1,3,3.00,2,1.00,NaN,1 en aparcamiento/garaje público,1.00,1.00,NaN,Individual,2010.0,NaN,NaN,Sin jardín,NaN,NaN,NaN,NaN,203.00,NaN,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0.00,0.00,NaN,0.00,1.00,0.00,0.00,0.00,"Bonito piso en el Ensanche, cerca del metro y todos los sservicios. Cerca del colegio Stella Maris, metro la gavia y cc.la gavia. Bien comunicado con la A3. Buen estado de conservación.",NaN,NaN,NaN,Indomio.com,2024-11-20,No indicado,-,0.00,NaN,183.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sin_informacion,Desconocido
19,"Piso de dos habitaciones buen estado, San Diego, Madrid",https://www.indomio.es/anuncios/88009171/,210000.00,NaN,NaN,2.80,75.00,Puente de Vallecas,13.00,SanDiego,132.00,San Diego,NaN,Piso de dos

In [1205]:
df17['tipo_inmueble'].value_counts(dropna=False)

tipo_inmueble
NaN          9959
económico     276
medio         173
señorial      121
Name: count, dtype: int64

In [1206]:
df17['tiene_armario'].value_counts(dropna=False)

tiene_armario
NaN     9811
1.00     718
Name: count, dtype: int64

In [1207]:
df17['cancha_tenis'].value_counts(dropna=False)

cancha_tenis
NaN     7091
0.00    3435
1.00       3
Name: count, dtype: int64

In [1208]:
df17['chimenea'].value_counts(dropna=False)

chimenea
NaN     6209
0.00    4268
1.00      52
Name: count, dtype: int64

In [1209]:
df17[df17['chimenea'] == 1][['calefaccion', 'chimenea']]

,calefaccion,chimenea
22,NaN,1.00
81,NaN,1.00
208,"Central, por radiadores, de gas",1.00
212,"Individual, por radiadores, de gas",1.00
227,NaN,1.00
...,...,...
9581,Individual,1.00
9746,1.0,1.00
9749,1.0,1.00
9750,1.0,1.00


In [1210]:
df17['fibra_optica'].value_counts(dropna=False)

fibra_optica
NaN     7091
0.00    3396
1.00      42
Name: count, dtype: int64

In [1211]:
df17['tiene_piscina'].value_counts(dropna=False)

tiene_piscina
NaN     9425
1.00    1104
Name: count, dtype: int64

In [1212]:
df17['carpinteria_exterior_doble_vidrio/pvc'].value_counts(dropna=False)

carpinteria_exterior_doble_vidrio/pvc
NaN     7091
0.00    3419
1.00      19
Name: count, dtype: int64

In [1213]:
df17[df17['tipo_inmueble'].notnull()]

,title,url,precio,precio_anterior,descuento,EUR/m2,m2_constr,distrito,cod_distrito,barrio,cod_barrio,zona,calle,detalle,consumoce_ano,letrace,emisiones_co2,tipologia,propiedad_completa,estado,amueblado,planta,habitaciones,dormitorios,banos,balcon,terraza,garaje,trastero,ascensor,porteria,calefaccion,ano_construccion,plantas_edificio,aire_acondicionado,jardin,inmueble_ingresos,alquiler_opcion_a_compra,disponibilidad,nuda_propiedad,sup_comercial,tipo_inmueble,tiene_armario,cancha_tenis,carpinteria_exterior_doble_vidrio/pvc,carpinteria_exterior_doble_vidrio/madera,carpinteria_exterior_doble_vidrio/metal,carpinteria_exterior_triple_vidrio/madera,carpinteria_exterior_triple_vidrio/metal,carpinteria_exterior_vidrio/pvc,carpinteria_exterior_vidrio/madera,carpinteria_exterior_vidrio/metal,chimenea,cocina,fibra_optica,instalacion_tv_centralizada,instalacion_tv_individual,exterior,interior,interior_y_exterior,tiene_piscina,porton_electrico,puerta_blindada,alarma,videoportero,lujosa,descripcion_extendida,€_comunidad_mes,acceso_minusvalido,orientacion,web,fecha_descarga,certificado_energetico,piscina,tiene_jardin,barrios,cod_bario,referencia,cantidad_visitas,fecha_publicacion,titular_anuncio,campo12,cantidad_armarios,consumo_energia,cal_energetica,agua_caliente,tipo_suelo,m2_utiles,vista_zona,gastos_comunidad,zonas_verdes,terrazas,puerta_seguridad,num_plantas,num_pisos,reservado,nombre_oficina,cod_barrio_2,codigo_Postal,tipo_fachada,cantidad_dormitorios,orientacion_este,orientacion_norte,"orientacion_norte,_sur",orientacion_oeste,orientacion_sur,"orientacion_sur,_este",sistema_alarma,n.visitas,grupo_cocina,clasificacion_aire
1,"Piso de dos habitaciones buen estado, entreplanta, Casco Histórico de Barajas, Madrid",https://www.indomio.es/anuncios/79178265/,287000.00,NaN,NaN,3.46,83.00,Barajas,21.00,CascoHistóricodeBarajas,213.00,Casco Histórico de Barajas,NaN,"Piso de dos habitaciones buen estado, entreplanta",desconocido,NaN,No indicado,Piso,1.00,Bueno / Habitable,Sí,Entreplanta,2,2.00,2,0.00,1.00,1 en aparcamiento/garaje público,0.00,1.00,NaN,"Individual, por radiadores, alimentación eléctrica",2009.0,2.00,Preinstalación,NaN,1.00,NaN,NaN,NaN,NaN,medio,NaN,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Cocina abierta,1.00,1.00,0.00,0.00,1.00,0.00,NaN,0.00,1.00,0.00,0.00,0.00,"En Finca de 15 años vendemos 2 viviendas en entreplanta, se dividieron en Obra Nueva, tienen una única Referencia Catastral por lo que se vende como una sola vivienda. Pero son dos alquileres, totalmente independientes, siendo: (Apartamento de 1 dormitorio con baño y un Estudio con cocina independiente y baño) ambos amueblados con 2 puertas de acceso blindadas a descansillo y Contadores de agua y luz para cada vivienda. Con un único recibo de comunidad 123€ y un solo IBI al año.El Estudio se encuentra alquilado de larga duración y el apartamento se está preparando para alquilar de nuevo, pero ahora está vacío por si se quiere vivir en él o buscar inquilino a gusto del comprador. Rentabilidad garantizada. Dan a patio de la finca con derecho de uso de 22 metros, pero no tiene acceso directo.Su distribución es la siguiente: El Apartamento es amplio de un dormitorio, baño con ducha y salón-comedor con cocina integrada mas zona de almacenaje.El Estudio tiene pasillo ancho de entrada a zona diáfana sin dormitorio pero con cocina independiente y baño grande con ducha.Tienen preinstalación de aire acondicionado cada uno, radiadores eléctricos de energía azul. El agua caliente es por paneles solares centralizados, paredes en liso y suelos de gres, puertas en roble, techos de 2,65 lo que les da una gran amplitud. Son luminosos con sol de tarde con grandes ventanales de 2 hojas con rotura de puente térmico y climalit. Persianas térmicas. Puertas Blindadas.La finca es muy tranquila a 200 metros de Plaza de Barajas y del metro a 7 minutos, buenas calidades vivienda en entreplanta pero sin barreras arquitectónicas con ascensor incluso a la entreplanta y al garaje.Garaje Opcional

## cocina

In [1214]:
df17['cocina'].value_counts(dropna=False)

cocina
NaN                                   8927
Cocina equipada                        590
Cocina independiente                   323
Cocina amueblada                       190
Independiente                          131
                                      ... 
Equipada con excelentes calidades        1
Con entrada independiente                1
Independiente amueblada y equipada       1
Independiente con despensa               1
Amueblada reformada                      1
Name: count, Length: 88, dtype: int64

In [1215]:
df17['cocina'].unique()

array([nan, 'Cocina abierta', 'Cocina independiente',
       'Cocina abierta amueblada', 'Cocina independiente amueblada',
       'Cocina americana', 'Amueblada con electrodomésticos',
       'Cocina equipada', 'Americana', 'Independiente y equipada',
       'Amueblada sin electrodomésticos', 'Cocina amueblada',
       'Independiente', 'Amueblada, equipada', 'Individual',
       'Amueblada y en isla', 'Equipada', 'Amueblada',
       'Independiente con zona de servicio',
       'Independiente - amueblada con elec.', 'Independiente con office',
       'Amplia', 'Amueblada y equipada con electrodomésticos',
       'Pequeña cocina integrada', 'Americana, amueblada y equipada.',
       'Amueblada con electrodomésticos, caldera nueva',
       'Amueblada con electrodomesticos',
       '0ffice,habitacion ,baño y terraza',
       'Completamente equipada abierta al salón',
       'Independiente con office, dormi. Servic. Con baño',
       'En office amplia completamente equipada',
       'Co ofi

In [1216]:
df17.query("amueblado=='Sólo cocina amueblada' and cocina.notnull()")[['amueblado','cocina']]

,amueblado,cocina
493,Sólo cocina amueblada,Cocina abierta amueblada
1286,Sólo cocina amueblada,Cocina independiente amueblada
1755,Sólo cocina amueblada,Cocina independiente amueblada
1806,Sólo cocina amueblada,Cocina independiente amueblada
2831,Sólo cocina amueblada,Cocina independiente amueblada
9558,Sólo cocina amueblada,Cocina independiente amueblada


In [1217]:
#pongo el valor de cocina_1 en cocina y elimino el campo cocina_1
#df17.loc[df17['cocina_1'].notnull(), 'cocina'] = df17['cocina_1']
#el campo cocina_equipada que aparece en redpiso lo integro en cocina y luebo elimino el campo
#df17.loc[df17['cocina_equipada']==1, 'cocina'] = 'Cocina equipada'
df17.loc[df17['cocina']=='Francesa', 'cocina'] = 'independiente y amueblada'
df17.loc[(df17['amueblado'] == 'Sólo cocina amueblada') & (df17['cocina'].notnull()), 'cocina'] = df17['cocina'] + ' amueblada'
df17.loc[df17['cocina']=='Equipafda americana', 'cocina'] = 'Equipada americana'

#df17.drop(columns=['cocina_1'], inplace=True)
#df17.drop(columns=['cocina_equipada'], inplace=True)


In [1218]:
def agrupar_cocinas(tipo):
    categorias = []
    if pd.isnull(tipo):
        categorias.append('sin_informacion')
    elif isinstance(tipo, str):
        if 'abierta' in tipo.lower() or 'abierto' in tipo.lower() or 'americana' in tipo.lower() or 'integrada' in tipo.lower():
            categorias.append('abierta')
        if 'independiente' in tipo.lower():
            categorias.append('independiente')
        if 'amplia' in tipo.lower():
            categorias.append('amplia')
        if 'isla' in tipo.lower():
            categorias.append('con isla')
        if 'amueblada' in tipo.lower():
            categorias.append('amueblada')
        if 'con electrodomesticos' in tipo.lower() or 'equipada' in tipo.lower() or 'con elec.' in tipo.lower():
            categorias.append('equipada')       
        if 'sin amueblar' in tipo.lower():
            categorias.append('no amueblada')
        if 'reformada' in tipo.lower():
            return 'reformada'
        elif 'a reformar' in tipo.lower() or 'necesita reforma' in tipo.lower():
            return 'Sin reforma'
        if not categorias:
            categorias.append('sin categoria')
        return ', '.join(categorias)
    else:
        return 'error tipo campo no texto'
    return ', '.join(categorias)


In [1219]:
# Aplica la función de agrupación
df17['grupo_cocina'] = df17['cocina'].apply(agrupar_cocinas)

print(df17[['cocina', 'grupo_cocina']])


               cocina     grupo_cocina
0                 NaN  sin_informacion
1      Cocina abierta          abierta
2                 NaN  sin_informacion
3                 NaN  sin_informacion
4                 NaN  sin_informacion
...               ...              ...
10524             NaN  sin_informacion
10525             NaN  sin_informacion
10526             NaN  sin_informacion
10527             NaN  sin_informacion
10528             NaN  sin_informacion

[10529 rows x 2 columns]


In [1220]:
df17.query("cocina.notnull()")['cocina'].unique()

array(['Cocina abierta', 'Cocina independiente',
       'Cocina abierta amueblada amueblada',
       'Cocina independiente amueblada amueblada', 'Cocina americana',
       'Amueblada con electrodomésticos', 'Cocina equipada', 'Americana',
       'Independiente y equipada', 'Amueblada sin electrodomésticos',
       'Cocina amueblada', 'Independiente', 'Amueblada, equipada',
       'Individual', 'Amueblada y en isla', 'Equipada', 'Amueblada',
       'Independiente con zona de servicio',
       'Independiente - amueblada con elec.', 'Independiente con office',
       'Amplia', 'Amueblada y equipada con electrodomésticos',
       'Pequeña cocina integrada', 'Americana, amueblada y equipada.',
       'Amueblada con electrodomésticos, caldera nueva',
       'Amueblada con electrodomesticos',
       '0ffice,habitacion ,baño y terraza',
       'Completamente equipada abierta al salón',
       'Independiente con office, dormi. Servic. Con baño',
       'En office amplia completamente equipada',

In [1221]:
df17.query("amueblado=='Sólo cocina amueblada'").count()

title                    78
url                      78
precio                   78
precio_anterior           0
descuento                 0
                         ..
orientacion_sur,_este     0
sistema_alarma            5
n.visitas                 0
grupo_cocina             78
clasificacion_aire       78
Length: 111, dtype: int64

In [1222]:
df17['amueblado'].value_counts(dropna=False)

amueblado
NaN                       7606
0                         1469
0.0                        721
Sí                         305
No                         175
1.0                        161
Sólo cocina amueblada       78
Parcialmente amueblado      13
1                            1
Name: count, dtype: int64

In [1223]:
## interior, exterior
df17['interior'].value_counts(dropna=False)

interior
0.00    7785
NaN     2651
1.00      93
Name: count, dtype: int64

In [1224]:

df17['exterior'].value_counts(dropna=False)

exterior
0.00    7479
1.00    1815
NaN     1235
Name: count, dtype: int64

In [1225]:
df17["vista_zona"].unique()

array([nan, 'Exterior'], dtype=object)

In [1226]:
# Si el valor de la columna 'exterior' es nulo y la columna 'vista_zona' tiene el valor 'Exterior', poner en la columna 'exterior' un 1
df17.loc[df17['exterior'].isnull() & (df17['vista_zona'] == 'Exterior'), 'exterior'] = 1

# Eliminar la columna 'vista_zona'
df17.drop(columns=['vista_zona'], inplace=True)

In [1227]:

df17['interior_y_exterior'].value_counts(dropna=False)

interior_y_exterior
NaN     7091
0.00    3429
1.00       9
Name: count, dtype: int64

In [1228]:

df17.query("interior.isnull() & exterior.isnull() & interior_y_exterior.isnull()")["web"].unique()

array(['redpiso', 'pisosmadrid'], dtype=object)

## piscina

In [1229]:


df17['tiene_piscina'].value_counts(dropna=False)

tiene_piscina
NaN     9425
1.00    1104
Name: count, dtype: int64

In [1230]:
df17['piscina'].value_counts(dropna=False)

piscina
 NaN                                          4811
 -                                            3150
0.00                                          1896
 0                                             310
1.00                                           142
 Comunitaria                                   139
 Propia                                         75
 1                                               4
 Con salorium                                    1
 1 piscina de adultos y 1 piscina de niños       1
Name: count, dtype: int64

In [1231]:

#asignamos el valor tiene_piscina sin indicar el tipo
df17.loc[(df17['piscina'] != 0.0) & (df17['piscina'] != 0) & (df17['piscina'] != '-') & (df17['piscina'].notnull()), 'tiene_piscina'] = 1

In [1232]:
df17['tiene_piscina'].value_counts(dropna=False)

tiene_piscina
NaN     9425
1.00    1104
Name: count, dtype: int64

In [1233]:

df17.query("tiene_piscina.notnull()")["web"].unique()

array(['Indomio.com', 'pisos.com', 'redpiso', 'pisosmadrid'], dtype=object)

## porton electrico, puerta blindada

In [1234]:

## portn_electrico,puerta_blindad

columnas = ["porton_electrico", "puerta_blindada", "alarma","videoportero","lujosa","descripcion_extendida"]  
porcentaje_resultados = porcentaje_nulos(df17, columnas)


# Mostrar los resultados
for columna, porcentaje in porcentaje_resultados.items():
    print(f"Porcentaje de nulos en {columna}: {porcentaje:.2f}%")

Porcentaje de nulos en porton_electrico: 67.35%
Porcentaje de nulos en puerta_blindada: 67.35%
Porcentaje de nulos en alarma: 47.74%
Porcentaje de nulos en videoportero: 58.97%
Porcentaje de nulos en lujosa: 67.35%
Porcentaje de nulos en descripcion_extendida: 45.42%


In [1235]:

df17['porton_electrico'].value_counts(dropna=False)

porton_electrico
NaN     7091
0.00    3421
1.00      17
Name: count, dtype: int64

In [1236]:

df17['puerta_blindada'].value_counts(dropna=False)

puerta_blindada
NaN     7091
0.00    3337
1.00     101
Name: count, dtype: int64

In [1237]:

df17['alarma'].value_counts(dropna=False)

alarma
0.00    5409
NaN     5027
1.00      93
Name: count, dtype: int64

In [1238]:

df17.query("alarma==1 | puerta_blindada==1")[['precio','alarma','puerta_blindada']]

,precio,alarma,puerta_blindada
0,2000000.00,0.00,1.00
1,287000.00,0.00,1.00
18,450000.00,0.00,1.00
52,199990.00,0.00,1.00
79,176000.00,0.00,1.00
...,...,...,...
10422,1939000,1.00,NaN
10469,800000,1.00,NaN
10508,870000,1.00,NaN
10509,515000,1.00,NaN


In [1239]:

df17['videoportero'].value_counts(dropna=False)

videoportero
NaN     6209
0.00    3969
1.00     351
Name: count, dtype: int64

In [1240]:
## lujosa

In [1241]:

df17['lujosa'].value_counts(dropna=False)

lujosa
NaN     7091
0.00    1872
1.00    1566
Name: count, dtype: int64

In [1242]:

df17.query("videoportero==1").count()

title                    351
url                      351
precio                   351
precio_anterior            0
descuento                  0
                        ... 
orientacion_sur,_este      0
sistema_alarma             1
n.visitas                282
grupo_cocina             351
clasificacion_aire       351
Length: 110, dtype: int64

## descripcion_extendida

################ REVISAR #################

€_comunidad_mes

In [1243]:
df17['€_comunidad_mes'].value_counts(dropna=False)

€_comunidad_mes
NaN                 9191
Más de 100 €         350
Entre 40 y 60 €      100
Entre 80 y 100 €      95
Entre 60 y 80 €       80
                    ... 
336                    1
175                    1
325                    1
525                    1
60 €.-                 1
Name: count, Length: 314, dtype: int64

In [1244]:
df17['gastos_comunidad'].value_counts(dropna=False)

gastos_comunidad
NaN               10436
Comunidad 5000        9
€ 50/mes              6
€ 100/mes             5
Comunidad 4500        4
                  ...  
€ 138/mes             1
€ 90/mes              1
€ 120/mes             1
€ 61/mes              1
€ 140/mes             1
Name: count, Length: 58, dtype: int64

In [1245]:
df17.shape

(10529, 110)

In [1246]:
df17.query("`€_comunidad_mes`.notnull() | gastos_comunidad.notnull()")[['web','gastos_comunidad','€_comunidad_mes']]

,web,gastos_comunidad,€_comunidad_mes
0,Indomio.com,NaN,100
1,Indomio.com,NaN,123
11,Indomio.com,NaN,55
12,Indomio.com,NaN,60
13,Indomio.com,NaN,60
...,...,...,...
9616,Indomio.com,€ 100/mes,NaN
9620,Indomio.com,€ 100/mes,NaN
9621,Indomio.com,€ 140/mes,NaN
9623,Indomio.com,€ 50/mes,NaN


In [1247]:
df17.query("`€_comunidad_mes`.notnull() and  gastos_comunidad.isnull() ")[['web','gastos_comunidad','€_comunidad_mes']]

,web,gastos_comunidad,€_comunidad_mes
0,Indomio.com,NaN,100
1,Indomio.com,NaN,123
11,Indomio.com,NaN,55
12,Indomio.com,NaN,60
13,Indomio.com,NaN,60
...,...,...,...
7866,pisos.com,NaN,Más de 100 €
7873,pisos.com,NaN,Más de 100 €
7874,pisos.com,NaN,38
7876,pisos.com,NaN,Entre 20 y 40 €


In [1248]:
df17.query(" gastos_comunidad.notnull() and `€_comunidad_mes`.isnull() ")[['web','url','gastos_comunidad','€_comunidad_mes']]

,web,url,gastos_comunidad,€_comunidad_mes
7892,redpiso,https://www.redpiso.es/inmueble/piso-en-venta-en-los-rosales-villaverde-madrid-madrid-RP2372023110992,Comunidad 7400,NaN
7914,redpiso,https://www.redpiso.es/inmueble/piso-en-venta-en-villaverde-madrid-madrid-RP2372024127521,Comunidad 4000,NaN
7976,redpiso,https://www.redpiso.es/inmueble/atico-en-venta-en-calle-san-clodoaldo-ventas-ciudad-lineal-madrid-madrid-RP162024125933,Comunidad 9900,NaN
8030,redpiso,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-godella-san-cristobal-villaverde-madrid-madrid-RP2842024125361,Comunidad 5000,NaN
8037,redpiso,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-almendrales-almendrales-usera-madrid-madrid-RP392024129914,Comunidad 2000,NaN
...,...,...,...,...
9616,Indomio.com,https://www.indomio.es/anuncios/96986027/,€ 100/mes,NaN
9620,Indomio.com,https://www.indomio.es/anuncios/96967621/,€ 100/mes,NaN
9621,Indomio.com,https://www.indomio.es/anuncios/96979909/,€ 140/mes,NaN
9623,Indomio.com,https://www.indomio.es/anuncios/96696025/,€ 50/mes,NaN


In [1249]:
#En redpiso los valores hay que dividirlos entre 100 al procesarlo se quito la coma que tenia en 20,00 ahora aparece con 2000
#pisos madrid y no tiene gastos de comunidad

In [1250]:
df17.gastos_comunidad.unique()

array([nan, 'Comunidad 7400', 'Comunidad 4000', 'Comunidad 9900',
       'Comunidad 5000', 'Comunidad 2000', 'Comunidad 4500',
       'Comunidad 18000', 'Comunidad 10000', 'Comunidad 11000',
       'Comunidad 6500', 'Comunidad 3500', 'Comunidad 15000',
       'Comunidad 6900', 'Comunidad 3000', 'Comunidad 3750',
       'Comunidad 23600', 'Comunidad 6000', 'Comunidad 38000',
       '€ 175/mes', '€ 50/mes', '€ 37/mes', '€ 35/mes', '€ 112/mes',
       '€ 457/mes', '€ 95/mes', '€ 40/mes', '€ 111/mes', '€ 148/mes',
       '€ 160/mes', '€ 80/mes', '€ 138/mes', '€ 90/mes', '€ 120/mes',
       '€ 61/mes', '€ 451/mes', '€ 350/mes', '€ 43/mes', '€ 230/mes',
       '€ 125/mes', '€ 55/mes', '€ 100/mes', '€ 72/mes', '€ 58/mes',
       '€ 283/mes', '€ 27/mes', '€ 33/mes', '€ 26/mes', '€ 103/mes',
       '€ 163/mes', '€ 60/mes', '€ 30/mes', '€ 180/mes', '€ 280/mes',
       '€ 404/mes', '€ 109/mes', '€ 45/mes', '€ 140/mes'], dtype=object)

In [1251]:
# Sustituir las palabras 'Comunidad', '€', y '/mes' por vacío en 'gastos_comunidad'
df17['gastos_comunidad'] = df17['gastos_comunidad'].str.replace('Comunidad', '')
df17['gastos_comunidad'] = df17['gastos_comunidad'].str.replace('€', '')
df17['gastos_comunidad'] = df17['gastos_comunidad'].str.replace('/mes', '')
# Eliminar espacios en blanco adicionales
df17['gastos_comunidad'] = df17['gastos_comunidad'].str.strip()
 #Convertir solo los valores no nulos de 'gastos_comunidad' a tipo float
df17.loc[df17['gastos_comunidad'].notnull(), 'gastos_comunidad'] = df17.loc[df17['gastos_comunidad'].notnull(), 'gastos_comunidad'].astype(float)
# Dividir el valor de 'gastos_comunidad' entre 100 cuando 'web' es igual a 'redpiso'
df17.loc[(df17['web'] == 'redpiso') & (df17['gastos_comunidad'].notnull()), 'gastos_comunidad'] /= 100

# Dividir el valor de 'gastos_comunidad' entre 100 cuando 'web' es igual a 'redpiso'
# Convertir la columna 'gastos_comunidad' a tipo float
#df17['gastos_comunidad'] = pd.to_numeric(df17['gastos_comunidad'], errors='coerce')
#df17.loc[df17['web'] == 'redpiso', 'gastos_comunidad'] /= 100

In [1252]:
df17.gastos_comunidad.unique()

array([nan, 74.0, 40.0, 99.0, 50.0, 20.0, 45.0, 180.0, 100.0, 110.0, 65.0,
       35.0, 150.0, 69.0, 30.0, 37.5, 236.0, 60.0, 380.0, 175.0, 37.0,
       112.0, 457.0, 95.0, 111.0, 148.0, 160.0, 80.0, 138.0, 90.0, 120.0,
       61.0, 451.0, 350.0, 43.0, 230.0, 125.0, 55.0, 72.0, 58.0, 283.0,
       27.0, 33.0, 26.0, 103.0, 163.0, 280.0, 404.0, 109.0, 140.0],
      dtype=object)

In [1253]:
# Obtener los valores únicos de la columna '€_comunidad_mes'
df17['€_comunidad_mes'].unique()


array(['100', '123', nan, '55', '60', '258', '50', '92', '27', '90',
       '132', '128', '113', '800', '180', '1.300', '65', '625', '34',
       '150', '308', '146', 'Ningún gasto de comunidad', '310', '350',
       '110', '96', '120', '210', '70', '1.000', '250', '84', '247', '3',
       '56', '450', '40', '200', '75', '190', '166', '15', '45', '228',
       '72', '9', '294', '30', '160', '430', '95', '104', '63', '13',
       '85', '119', '300', '168', '340', '99', '164', '130', '241', '167',
       '5', '33', '111', '420', '206', '500', '10', '14', '540', '750',
       '148', '147', '386', '17', '76', '230', '205', '46', '47', '400',
       '32', '12', '36', '82', '155', '303', '207', '486', '83', '80',
       '66', '98', '178', '140', '52', '64', '54', '42', '29', '88', '61',
       '1.600', '440', '170', '57', '105', '142', '255', '125', '600',
       '324', '67', '269', '285', '38', '39', '102', '270', '44', '103',
       '617', '552', '311', '457', '192', '327', '288', '316', '

In [1254]:
# Usar comillas invertidas para nombres de columnas con caracteres especiales
df17.query("`€_comunidad_mes` == 'Comunidad: 320€ mensual  / ibi 1.040,96€ anual'")[["web"]]

,web
4865,pisos.com


#############revisar

#Tengo que ir al origen para separar los valores de gastos de comunidad y de ibi

In [1255]:
# Reemplazar valores nulos en 'gastos_comunidad' con los valores de '€_comunidad_mes'
df17['gastos_comunidad'] = df17['gastos_comunidad'].fillna(df17['€_comunidad_mes'])
# Eliminar la columna '€_comunidad_mes'
df17.drop(columns=['€_comunidad_mes'], inplace=True)

In [1256]:

# Reemplazar 'No tiene' y 'Ningún gasto de comunidad' por '0'
df17['gastos_comunidad'] = df17['gastos_comunidad'].replace({'No tiene': '0', 'Ningún gasto de comunidad': '0'})


In [1257]:
df17['acceso_minusvalido'].value_counts(dropna=False)

acceso_minusvalido
 NaN    8794
 0      1146
0.00     498
1.00      69
 1        10
 Sí        7
 No        5
Name: count, dtype: int64

# acceso_minusvalidos

In [1258]:
# Reemplazar 'Sí' por 1 y 'No' por 0 en la columna 'acceso_minusvalido'
df17['acceso_minusvalido'] = df17['acceso_minusvalido'].replace({'Sí': 1, 'No': 0})

# Renombrar la columna 'acceso_minusvalido' a 'acceso_discapacidad'
df17.rename(columns={'acceso_minusvalido': 'acceso_discapacidad'}, inplace=True)

#orientacion

In [1259]:
df17["orientacion"].value_counts()

orientacion
Sur                  312
Este                 263
Orientacion Sur      162
Orientacion Este     154
Oeste                148
                    ... 
Este-oeste             1
Norte y este           1
Oeste y este           1
Sur, oeste             1
Sur, este y norte      1
Name: count, Length: 40, dtype: int64

In [1269]:
# Eliminar 'Orientacion' y limpiar espacios adicionales
df17['orientacion'] = df17['orientacion'].str.replace('Orientacion', '').str.strip()

# Sustituir ',' y '-' por '_' y poner los valores en minúsculas en la columna 'orientacion'
df17['orientacion'] = df17['orientacion'].str.replace(',', '_').str.replace('/', '_').str.replace('|', '_').str.replace('y', '_').str.replace('-', '_').str.lower()

# Eliminar espacios adicionales entre palabras
df17['orientacion'] = df17['orientacion'].str.replace(r'\s+', '_', regex=True).str.strip()

# Reemplazar múltiples valores en la columna 'orientacion' por np.nan
df17['orientacion'] = df17['orientacion'].replace({
    '__':'_',
    '_1': np.nan,
    'medio_día': np.nan,
    'sur_en_la_terraza__dormitorios': np.nan,
    'vistas_al_palacio_de_parcent.': np.nan,
    'varias':np.nan
})

df17['orientacion'] = df17['orientacion'].str.replace('__', '_')

In [1272]:
# Obtener los valores únicos de la columna 'orientacion' donde los valores no son nulos
df17.loc[df17['orientacion'].notnull(), 'orientacion'].unique()


array(['norte', 'sur_este', 'este', 'oeste', 'sur', 'norte_sur',
       'noroeste', 'sureste', 'suroeste', 'noreste', 'nordeste',
       'a_la_montaña', 'este_oeste', 'varias', 'sur_este_norte',
       'este_noreste_suroeste', 'sur_oeste', 'oeste_este', 'norte_este',
       'este_oeste_norte', 'sudeste', 'sur_norte'], dtype=object)

In [1271]:
df17['orientacion'] = df17['orientacion'].str.replace('__', '_')


In [ ]:
#renombrar las columnas que hay de orientacion para integranlas con orientacion y sus valores

In [366]:
#Revisar que filas no tiene descripcion_extendida , deberian tenerlas todas
df17.query("descripcion_extendida.isnull()")['web'].unique()

array(['Indomio.com', 'pisos.com', 'redpiso'], dtype=object)

In [367]:

df17.query("descripcion_extendida.notnull()")['web'].unique()

array(['Indomio.com', 'redpiso', 'pisosmadrid'], dtype=object)

In [ ]:






# Contar las filas donde 'dormitorios' es igual a 'cantidad_dormitorios'
conteo_filas_iguales = (df17['dormitorios'] == df17['cantidad_dormitorios']).sum()

print(conteo_filas_iguales)



In [42]:
df17.to_csv("dfpisosCompleto17012025.csv",index='False')

In [32]:
df17.columns.tolist()

['title',
 'url',
 'precio',
 'precio_anterior',
 'descuento',
 'EUR/m2',
 'm2_constr',
 'distrito',
 'cod_distrito',
 'barrio',
 'cod_barrio',
 'zona',
 'calle',
 'detalle',
 'consumoce_ano',
 'letrace',
 'emisiones_co2',
 'tipologia',
 'propiedad_completa',
 'estado',
 'amueblado',
 'planta',
 'habitaciones',
 'dormitorios',
 'banos',
 'balcon',
 'terraza',
 'garaje',
 'trastero',
 'ascensor',
 'porteria',
 'calefaccion',
 'ano_construccion',
 'plantas_edificio',
 'aire_acondicionado',
 'jardin',
 'inmueble_ingresos',
 'alquiler_opcion_a_compra',
 'disponibilidad',
 'nuda_propiedad',
 'sup_comercial',
 'tipo_inmueble',
 'tiene_armario',
 'cancha_tenis',
 'carpinteria_exterior_doble_vidrio/pvc',
 'carpinteria_exterior_doble_vidrio/madera',
 'carpinteria_exterior_doble_vidrio/metal',
 'carpinteria_exterior_triple_vidrio/madera',
 'carpinteria_exterior_triple_vidrio/metal',
 'carpinteria_exterior_vidrio/pvc',
 'carpinteria_exterior_vidrio/madera',
 'carpinteria_exterior_vidrio/metal',
 

In [46]:
# Contar las filas donde 'dormitorios' es igual a 'cantidad_dormitorios'
conteo_filas_iguales = (df17['dormitorios'] == df17['cantidad_dormitorios']).sum()

print(conteo_filas_iguales)


22


In [29]:
df17["cod_barrio"].value_conts(dropna=False)

AttributeError: 'Series' object has no attribute 'value_conts'